# Librerias necesarias para la extraccion de Data

In [ ]:
# Primero, instala la biblioteca de Google para la API de YouTube
!pip install --upgrade google-api-python-client
# Se agrega el formato de isodate para los filtros
!pip install isodate

# Se importan las librerias necesarias para el manejo de los datos
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime, timedelta
import json # Import the json module
import time
import isodate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 29.3 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.151.0
    Uninstalling google-api-python-client-2.151.0:
      Successfully uninstalled google-api-python-client-2.151.0


## Proteccion de la clave api

In [ ]:
# llamar a la api guardada en el colab
from google.colab import userdata


In [ ]:
# Clave de API
api_key = userdata.get('Apiyt')
# ID del canal de YouTube
channel_id = 'UCq2otJm7A6jGGcGFEks0HVw' # toca revisar en compartir y ver channel_id
# colocar nombre del camal
channel= 'Se regalan dudas'
# Inicializa el cliente de YouTube
youtube = build('youtube', 'v3', developerKey=api_key)

## Generacion codigo de extraccion api youtube V3

In [ ]:
# Función para obtener videos subidos por el canal
def get_videos_by_date_range(channel_id, start_date, end_date):
    video_data = []
    request = youtube.search().list(
        part="snippet",
        channelId=channel_id,
        maxResults=50,
        type="video",
        order="date",
        publishedAfter=start_date,
        publishedBefore=end_date
    )
    while request:
        response = request.execute()
        video_ids = [item['id']['videoId'] for item in response['items']]

        # Obtener detalles adicionales de los videos en lote
        if video_ids:
            video_details_request = youtube.videos().list(
                part="snippet,contentDetails,statistics",
                id=",".join(video_ids)
            )
            video_details_response = video_details_request.execute()

            for item in video_details_response['items']:
                video_id = item['id']
                title = item['snippet']['title']
                description = item['snippet']['description']
                duration = item['contentDetails']['duration']
                publish_date = item['snippet']['publishedAt']
                url = f"https://www.youtube.com/watch?v={video_id}"
                likes = item['statistics'].get('likeCount', 0)
                views = item['statistics'].get('viewCount', 0)
                C_comments = item['statistics'].get('commentCount', 0)

                # Convertir la duración a minutos
                duration_minutes = isodate.parse_duration(duration).total_seconds() / 60

                # Filtrar videos que duren más de 45 minutos
                if duration_minutes > 45:
                    video_data.append({
                        'video_id': video_id,
                        'title': title,
                        'description': description,
                        'duration': duration,
                        'publish_date': publish_date,
                        'url': url,
                        'likes': likes,
                        'views': views,
                        'C_comments': C_comments
                    })

        # Paginación para obtener más videos
        request = youtube.search().list_next(request, response)

    return video_data

### seleccion del año de interes e iteracion para que la api saque toda la data

In [ ]:
# Configurar los intervalos de fechas para dividir en rangos de 6 meses
def generate_date_ranges(start, end, delta=180):
    date_ranges = []
    current_date = start
    while current_date < end:
        next_date = current_date + timedelta(days=delta)
        date_ranges.append((current_date.isoformat() + 'Z', next_date.isoformat() + 'Z'))
        current_date = next_date
    return date_ranges

In [ ]:
# Fechas de inicio y fin de tu búsqueda
start_date = datetime(2018, 1, 1)  # Fecha de inicio
end_date = datetime.now()           # Fecha actual

In [ ]:
# Genera rangos de fechas cada 6 meses
date_ranges = generate_date_ranges(start_date, end_date)

### Seleccion de data del canal y se guarde en otro dataset

In [ ]:
# Función para obtener información básica del canal
def get_channel_info(channel_id):
    request = youtube.channels().list(
        part="snippet,statistics,brandingSettings",
        id=channel_id
    )
    response = request.execute()

    # Extraer información del canal
    if "items" in response and len(response["items"]) > 0:
        channel_info = response["items"][0]
        data = {
            "title": channel_info["snippet"]["title"],
            "description": channel_info["snippet"]["description"],
            "creation_date": channel_info["snippet"]["publishedAt"],
            "country": channel_info["snippet"].get("country", "No especificado"),
            "subscriber_count": int(channel_info["statistics"].get("subscriberCount", 0)),
            "view_count": int(channel_info["statistics"].get("viewCount", 0)),
            "video_count": int(channel_info["statistics"].get("videoCount", 0))
        }
        return data
    else:
        return None


### Extraccion de la data deacuerdo a al funcion planteada

In [ ]:
# Obtener información y almacenarla en un DataFrame
channel_info = get_channel_info(channel_id)
df_channel = pd.DataFrame([channel_info])

In [ ]:
df_channel

,title,description,creation_date,country,subscriber_count,view_count,video_count
0,Se Regalan Dudas Podcast,Somos Lety & Ash y juntas creamos Se Regalan D...,2018-08-21T01:44:22Z,MX,499000,38619991,1083


In [ ]:
# Obtener todos los videos subidos por el canal
video_ids = []
for start, end in date_ranges:
    try:
        videos = get_videos_by_date_range(channel_id, start, end)
        video_ids.extend(videos)
        time.sleep(1)  # Pausa para evitar llegar al límite de solicitudes
    except Exception as e:
        print(f"Error durante la extracción de datos: {e}")

In [ ]:
# Extraer comentarios de cada video y guardarlos en el dataframe

video_list = []  # Create an empty list to store video data
for video in video_ids:
    video_id = video['video_id']
    video_list.append({
    'video_id': video['video_id'],
    'title': video['title'],
    'description': video['description'],
    'duration': video['duration'],
    'publish_date': video['publish_date'],
    'url': video['url'],
    'likes': video['likes'],
    'views': video['views'],
    'Cantidad_comentarios': video['C_comments'],
    'canal': channel,
    'transcripcion': '',
    'estado': 'False'
})

# Create DataFrame from list of video data
df = pd.DataFrame(video_list)

In [ ]:
df

,video_id,title,description,duration,publish_date,url,likes,views,Cantidad_comentarios,canal,transcripcion,estado
0,-opyuUf4ha4,20. Salir del clóset | David Rodríguez,🎧Escuchar en Spotify → http://bit.ly/srdyoutub...,PT53M35S,2019-09-25T17:40:31Z,https://www.youtube.com/watch?v=-opyuUf4ha4,421,17586,12,Se regalan dudas,,False
1,QShQE5vmJ7c,21. Relaciones que no funcionan | Romina Sacre,🎧Escuchar Nueva temporada en Spotify → http://...,PT46M15S,2019-09-25T17:40:33Z,https://www.youtube.com/watch?v=QShQE5vmJ7c,3781,192562,186,Se regalan dudas,,False
2,za2Oj5UmlGQ,19. Conoce el eneagrama | Susana Basañez,🎧Escuchar en Spotify → http://bit.ly/srdyoutub...,PT56M7S,2019-09-25T17:39:19Z,https://www.youtube.com/watch?v=za2Oj5UmlGQ,644,32821,24,Se regalan dudas,,False
3,0VlPfqTZQEk,18. Cumpliendo mis propósitos | Juan Pablo God...,🎧Escuchar en Spotify → http://bit.ly/srdyoutub...,PT48M34S,2019-09-25T17:38:00Z,https://www.youtube.com/watch?v=0VlPfqTZQEk,219,10929,5,Se regalan dudas,,False
4,40AdBCn6A4w,11. Minimalismo. Ser feliz con menos | Eduardo...,🎧Escuchar en Spotify → http://bit.ly/srdyoutub...,PT48M30S,2019-09-25T17:35:39Z,https://www.youtube.com/watch?v=40AdBCn6A4w,497,24097,20,Se regalan dudas,,False
...,...,...,...,...,...,...,...,...,...,...,...,...
186,6PizPzgdFFI,470. How to spot a narcissist in a relationshi...,"In today’s episode, we invited Dr. Ramani, cli...",PT1H9M9S,2024-06-25T06:00:03Z,https://www.youtube.com/watch?v=6PizPzgdFFI,1299,34806,123,Se regalan dudas,,False
187,ANBeHKb3Kxo,468. Cómo viven los hombres el sexo | Roberto ...,¿Los hombres también tienen miedos e insegurid...,PT58M16S,2024-06-18T06:00:04Z,https://www.youtube.com/watch?v=ANBeHKb3Kxo,8066,334765,297,Se regalan dudas,,False
188,LYprOnwI_8Y,466. Miedos y retos de ser papá por primera ve...,Las expectativas que las personas sienten al c...,PT51M46S,2024-06-11T06:00:07Z,https://www.youtube.com/watch?v=LYprOnwI_8Y,737,17411,69,Se regalan dudas,,False
189,UaVagRihtXE,464. ¿Qué hago con mi corazón roto? | Elvira S...,En el episodio de hoy platicamos con Elvira Sa...,PT1H4M53S,2024-06-04T06:00:16Z,https://www.youtube.com/watch?v=UaVagRihtXE,2679,92291,121,Se regalan dudas,,False


In [ ]:
# Verificar los últimos resultados
df = df.sort_values(by='publish_date', ascending=False)
df.head()

,video_id,title,description,duration,publish_date,url,likes,views,Cantidad_comentarios,canal,transcripcion,estado
165,oQHvbX5uq6E,512. Rompiendo estereotipos: ¿Cómo viven los h...,¿Es difícil para los hombres conectar desde la...,PT1H10M45S,2024-11-19T06:00:35Z,https://www.youtube.com/watch?v=oQHvbX5uq6E,1031,40350,61,Se regalan dudas,,False
166,6oWKEBkcw-U,510. Claves para una relación duradera | Nilda...,¿Cuál es la clave para una relación duradera? ...,PT1H3M59S,2024-11-14T18:06:45Z,https://www.youtube.com/watch?v=6oWKEBkcw-U,1396,44552,98,Se regalan dudas,,False
167,bsMmr2n6P6c,Los favoritos de Ash | 🔴 Live 24/7,"Esta semana, quiero compartirles algunos de mi...",PT59M48S,2024-11-05T08:28:20Z,https://www.youtube.com/watch?v=bsMmr2n6P6c,27,1353,0,Se regalan dudas,,False
168,T5Bocl1L1ew,508. Secuestrado en una caja por 290 días | Al...,Esta es una de las historias más fuertes que t...,PT1H54M37S,2024-11-05T06:00:22Z,https://www.youtube.com/watch?v=T5Bocl1L1ew,789,32013,67,Se regalan dudas,,False
169,T3z-5kzcfzM,506. ‘Ahí no es': Cómo reconocer si una relaci...,¿Cómo saber cuándo ‘ahí no es’ en una relación...,PT1H22M49S,2024-10-29T06:00:25Z,https://www.youtube.com/watch?v=T3z-5kzcfzM,1757,63810,114,Se regalan dudas,,False


## Codigo para comentarios

In [ ]:
# Función para obtener comentarios de un video
def get_video_comments(video_id):
    comments = []
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        maxResults=100  # Máximo permitido por solicitud
    )

    while request:
        response = request.execute()

        for item in response["items"]:
            comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
            author = item["snippet"]["topLevelComment"]["snippet"]["authorDisplayName"]
            published_at = item["snippet"]["topLevelComment"]["snippet"]["publishedAt"]
            like_count = item["snippet"]["topLevelComment"]["snippet"]["likeCount"]

            comments.append({
                "video_id": video_id,
                "author": author,
                "comment": comment,
                "published_at": published_at,
                "like_count": like_count
            })

        # Paginación para obtener más comentarios
        request = youtube.commentThreads().list_next(request, response)

    return comments

In [ ]:
# Obtener el ID del primer video en el DataFrame
primer_video_id = df['video_id'].iloc[0]

# Obtener comentarios del primer video
print(f"Obteniendo comentarios del primer video ID: {primer_video_id}")
try:
    primer_video_comments = get_video_comments(primer_video_id)
    # Convertir los comentarios a un DataFrame
    primer_video_comments_df = pd.DataFrame(primer_video_comments)
except Exception as e:
    print(f"Error al obtener comentarios del primer video {primer_video_id}: {e}")

Obteniendo comentarios del primer video ID: rF_13DedbTU


In [ ]:
primer_video_comments_df

,video_id,author,comment,published_at,like_count
0,rF_13DedbTU,@RobertoMtzTV,El terremoto de 1985 en México marcó a toda un...,2024-11-07T22:06:48Z,11
1,rF_13DedbTU,@feratgonzalezrodrigo1517,Uff se antoja uno así del movimiento estudiant...,2024-11-08T12:17:48Z,0
2,rF_13DedbTU,@YZeppelin,"Este &quot;creativo&quot; es como el gusgri, w...",2024-11-08T08:46:53Z,0
3,rF_13DedbTU,@fredssm,shadowban xd,2024-11-08T07:51:32Z,0
4,rF_13DedbTU,@Jackadrenalina,Creí que era luna bella pelinegra. La srta may...,2024-11-08T07:33:00Z,0
5,rF_13DedbTU,@DUSO7512,"Acaba de estrenar la serie, llevo 3 capítulos,...",2024-11-08T07:28:35Z,0
6,rF_13DedbTU,@Pitacrumbs,PÍCALE AL LIKE SI CREES QUE ROBERTO DEBERÍA IR...,2024-11-08T06:42:49Z,0
7,rF_13DedbTU,@AusenteYo,denle una Maruchan...............................,2024-11-08T06:17:57Z,0
8,rF_13DedbTU,@javieramezcua2965,Y José Madero?,2024-11-08T03:55:11Z,1
9,rF_13DedbTU,@RangelEdgar-ct9in,"Mi padre Edgar Garcia es sobreviviente, tenemo...",2024-11-08T03:29:27Z,4


### Extraccion de comentarios para el canal y se deja en formato Json

In [ ]:
# Función para obtener comentarios en formato JSON
def get_video_comments_json(video_id):
    comments = []
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        maxResults=100  # Máximo permitido por solicitud
    )

    while request:
        response = request.execute()

        for item in response["items"]:
            comment_data = {
                "author": item["snippet"]["topLevelComment"]["snippet"]["authorDisplayName"],
                "comment": item["snippet"]["topLevelComment"]["snippet"]["textDisplay"],
                "published_at": item["snippet"]["topLevelComment"]["snippet"]["publishedAt"],
                "like_count": item["snippet"]["topLevelComment"]["snippet"]["likeCount"]
            }
            comments.append(comment_data)

        # Paginación para obtener más comentarios
        request = youtube.commentThreads().list_next(request, response)

    # Convertir lista de comentarios a JSON
    return json.dumps(comments)

In [ ]:
# Agregar columna de comentarios en formato JSON al DataFrame de videos
df['comments_json'] = None  # Crear columna vacía para los comentarios

for index, row in df.iterrows():
    video_id = row['video_id']
    print(f"Obteniendo comentarios del video ID: {video_id}")
    try:
        comments_json = get_video_comments_json(video_id)
        df.at[index, 'comments_json'] = comments_json
        time.sleep(2)  # Pausa para evitar límite de API
    except Exception as e:
        print(f"Error al obtener comentarios del video {video_id}: {e}")
        df.at[index, 'comments_json'] = json.dumps([])  # Asignar lista vacía si ocurre un error


Obteniendo comentarios del video ID: oQHvbX5uq6E
Obteniendo comentarios del video ID: 6oWKEBkcw-U
Obteniendo comentarios del video ID: bsMmr2n6P6c
Obteniendo comentarios del video ID: T5Bocl1L1ew
Obteniendo comentarios del video ID: T3z-5kzcfzM
Obteniendo comentarios del video ID: 8PplaRq0wi4
Obteniendo comentarios del video ID: 9kvMa_c7T58
Obteniendo comentarios del video ID: VVt7oe8hoP0
Obteniendo comentarios del video ID: D4eL_Tf1mu4
Obteniendo comentarios del video ID: xB8xROo0nS4
Obteniendo comentarios del video ID: tUidteIXH6Q
Obteniendo comentarios del video ID: I9yzqHd7OVg
Obteniendo comentarios del video ID: eFZXxj1akZE
Obteniendo comentarios del video ID: IVTRGHr_Ahc
Obteniendo comentarios del video ID: D7GQvsaqaoE
Obteniendo comentarios del video ID: 1wb_vr4tSsU
Obteniendo comentarios del video ID: Njah_bqrEFs
Obteniendo comentarios del video ID: 1rG2JYdgEc8
Obteniendo comentarios del video ID: Bn5GaqorSVI
Obteniendo comentarios del video ID: -nq0lDoCvfY
Obteniendo comentari

In [ ]:
df

,video_id,title,description,duration,publish_date,url,likes,views,Cantidad_comentarios,canal,transcripcion,estado,comments_json
165,oQHvbX5uq6E,512. Rompiendo estereotipos: ¿Cómo viven los h...,¿Es difícil para los hombres conectar desde la...,PT1H10M45S,2024-11-19T06:00:35Z,https://www.youtube.com/watch?v=oQHvbX5uq6E,1031,40350,61,Se regalan dudas,,False,"[{""author"": ""@brendapechquintana8070"", ""commen..."
166,6oWKEBkcw-U,510. Claves para una relación duradera | Nilda...,¿Cuál es la clave para una relación duradera? ...,PT1H3M59S,2024-11-14T18:06:45Z,https://www.youtube.com/watch?v=6oWKEBkcw-U,1396,44552,98,Se regalan dudas,,False,"[{""author"": ""@seregalandudas"", ""comment"": ""Ent..."
167,bsMmr2n6P6c,Los favoritos de Ash | 🔴 Live 24/7,"Esta semana, quiero compartirles algunos de mi...",PT59M48S,2024-11-05T08:28:20Z,https://www.youtube.com/watch?v=bsMmr2n6P6c,27,1353,0,Se regalan dudas,,False,[]
168,T5Bocl1L1ew,508. Secuestrado en una caja por 290 días | Al...,Esta es una de las historias más fuertes que t...,PT1H54M37S,2024-11-05T06:00:22Z,https://www.youtube.com/watch?v=T5Bocl1L1ew,789,32013,67,Se regalan dudas,,False,"[{""author"": ""@giovanamontenegro7614"", ""comment..."
169,T3z-5kzcfzM,506. ‘Ahí no es': Cómo reconocer si una relaci...,¿Cómo saber cuándo ‘ahí no es’ en una relación...,PT1H22M49S,2024-10-29T06:00:25Z,https://www.youtube.com/watch?v=T3z-5kzcfzM,1757,63810,114,Se regalan dudas,,False,"[{""author"": ""@seregalandudas"", ""comment"": ""Si ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,BsfnSZl5z2s,9. Miedo a crecer | Gloria Calzada,🎧Escuchar en Spotify → http://bit.ly/srdyoutub...,PT46M16S,2019-09-25T17:35:28Z,https://www.youtube.com/watch?v=BsfnSZl5z2s,983,45409,35,Se regalan dudas,,False,"[{""author"": ""@ginacorredorhernandez2268"", ""com..."
6,BMzUOg6Ix_s,8. Violencia doméstica | Maisa Covaleda,🎧Escuchar en Spotify → http://bit.ly/srdyoutub...,PT55M8S,2019-09-25T17:35:20Z,https://www.youtube.com/watch?v=BMzUOg6Ix_s,364,15569,20,Se regalan dudas,,False,"[{""author"": ""@CLAU_La_bibliofila"", ""comment"": ..."
12,83avrAkNsmY,7. El amor trasciende la vida | Lidia Zaragoza,🎧Escuchar en Spotify → http://bit.ly/srdyoutub...,PT50M49S,2019-09-25T17:35:16Z,https://www.youtube.com/watch?v=83avrAkNsmY,738,41085,28,Se regalan dudas,,False,"[{""author"": ""@JamesToncy"", ""comment"": ""hay gen..."
11,CVYi24dThNI,6. El amor de tu vida | Cherry Chris,🎧Escuchar en Spotify → http://bit.ly/srdyoutub...,PT51M15S,2019-09-25T17:35:08Z,https://www.youtube.com/watch?v=CVYi24dThNI,3988,208349,92,Se regalan dudas,,False,"[{""author"": ""@mariavaleriaferrandiz1238"", ""com..."


In [ ]:
# prompt: quiero saber de la columna comentarios quedaron  [] cuentame cuantos son

# Count the number of rows where 'comments_json' is an empty list
count_empty_comments = df['comments_json'].apply(lambda x: json.loads(x) == []).sum()

print(f"Número de filas con 'comments_json' como lista vacía: {count_empty_comments}")

Número de filas con 'comments_json' como lista vacía: 1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 10.2 MB/s eta 0:00:00


Conexión exitosa a MongoDB Atlas!


In [ ]:
# Función para obtener el tamaño de la base de datos
def check_db_size():
    stats = db.command("dbstats")
    db_size_in_mb = stats['storageSize'] / (1024 * 1024)  # Convertir a MB
    print(f"Tamaño de la base de datos: {db_size_in_mb:.2f} MB")
    # Umbral de 400 MB
    if db_size_in_mb > 400:
        print("Te pasaste")

In [ ]:
# Ejecutar la función para verificar el tamaño de la base de datos
check_db_size()

Tamaño de la base de datos: 31.76 MB


In [ ]:
# Convertir el DataFrame a un diccionario y hacer la inserción
data = df.to_dict(orient='records')  # 'records' convierte a una lista de diccionarios
collection.insert_many(data)

InsertManyResult([ObjectId('67426c8cb40baccea69a4877'), ObjectId('67426c8cb40baccea69a4878'), ObjectId('67426c8cb40baccea69a4879'), ObjectId('67426c8cb40baccea69a487a'), ObjectId('67426c8cb40baccea69a487b'), ObjectId('67426c8cb40baccea69a487c'), ObjectId('67426c8cb40baccea69a487d'), ObjectId('67426c8cb40baccea69a487e'), ObjectId('67426c8cb40baccea69a487f'), ObjectId('67426c8cb40baccea69a4880'), ObjectId('67426c8cb40baccea69a4881'), ObjectId('67426c8cb40baccea69a4882'), ObjectId('67426c8cb40baccea69a4883'), ObjectId('67426c8cb40baccea69a4884'), ObjectId('67426c8cb40baccea69a4885'), ObjectId('67426c8cb40baccea69a4886'), ObjectId('67426c8cb40baccea69a4887'), ObjectId('67426c8cb40baccea69a4888'), ObjectId('67426c8cb40baccea69a4889'), ObjectId('67426c8cb40baccea69a488a'), ObjectId('67426c8cb40baccea69a488b'), ObjectId('67426c8cb40baccea69a488c'), ObjectId('67426c8cb40baccea69a488d'), ObjectId('67426c8cb40baccea69a488e'), ObjectId('67426c8cb40baccea69a488f'), ObjectId('67426c8cb40baccea69a48

In [ ]:
# Ejecutar la función para verificar el tamaño de la base de datos
check_db_size()

Tamaño de la base de datos: 207.48 MB


In [ ]:
collection.count_documents({})

2266

In [ ]:
collection.find_one({'canal':'WORLDCA$T'})

{'_id': ObjectId('672f9928f8cfaa9f711f3ee7'),
 'video_id': 'AT18gQZTiC8',
 'title': '#145 - "Aniquilar a una Persona es algo que te Marca de por Vida" - Javier Cerezo',
 'description': '🎁 Clase gratis “Descubre la nueva profesión”: https://bit.ly/4eQuvCQ\n\nInstagram Javier Cerezo: https://www.instagram.com/simicabo2021/?hl=es\n\nEsperamos que lo disfruten!\n\nSíguenos en:\n\n-Spotify: https://open.spotify.com/show/5hiPtlv...\n\n\n-Tik tok: https://www.tiktok.com/@worldcast.oficial\n\n\n-Instagram: https://www.instagram.com/worldcast_/\n\n\nConoce más a Pedro Buerbaum:\n\n-Instagram: https://www.instagram.com/pedrobuerbaum/\n\n\nEmprendimiento, hábitos y negocios by Pedro Buerbaum en:\n\n-Youtube:   \n\nhttps://www.youtube.com/@PedroBuerbaum',
 'duration': 'PT1H50M3S',
 'publish_date': '2024-11-07T16:00:07Z',
 'url': 'https://www.youtube.com/watch?v=AT18gQZTiC8',
 'likes': '1089',
 'views': '43641',
 'Cantidad_comentarios': '92',
 'canal': 'WORLDCA$T',
 'transcripcion': '',
 'estado': 

In [ ]:
collection.distinct('canal')

['CHINOLO44',
 'Doctor Vic',
 'EL ANTIPODCAST',
 'El Dotol Nastra',
 'En terapia con Roberto rocha',
 'Gotitas de poliamor',
 'News To You',
 'PATABAJO El Podcast',
 'PICPOD TV',
 'Roberto Mz',
 'Sin Miedo Al Éxito',
 'Somos Pueblo Media',
 'Tengo un plan',
 'The Wild Project',
 'Uri Sabat',
 'VOS PODÉS, EL PODCAST!',
 'WORLDCA$T',
 'pepe&chema podcast']

In [ ]:
# Elimina los documentos con el campo `canal` igual a "tengo un plan"
collection.delete_many({"canal": "Tengo un plan"})


DeleteResult({'n': 1864, 'electionId': ObjectId('7fffffff00000000000000b1'), 'opTime': {'ts': Timestamp(1730940819, 1880), 't': 177}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1730940819, 1885), 'signature': {'hash': b'E\xd8\x14\xf9z\xb0Z\xc6uN|\xbe\r\xaf\x9d\xc2_\xb73x', 'keyId': 7376317674717970437}}, 'operationTime': Timestamp(1730940819, 1880)}, acknowledged=True)

In [ ]:
collection.count_documents({})

368

In [ ]:
collection.distinct('canal')

['CHINOLO44',
 'El Dotol Nastra',
 'PICPOD TV',
 'Red Pill Podcast',
 'Roberto Mz',
 'Sensibles y Chingonas',
 'Sin miedo al Exito',
 'Somos Pueblo Media',
 'Uri Sabat',
 'WORLDCA$T']

In [ ]:

# Obtén estadísticas de la base de datos
stats = db.command("dbstats")

In [ ]:
stats

{'db': 'TFM',
 'collections': 1,
 'views': 0,
 'objects': 2266,
 'avgObjSize': 196597.66813768755,
 'dataSize': 445490316,
 'storageSize': 217563136,
 'totalFreeStorageSize': 0,
 'numExtents': 0,
 'indexes': 1,
 'indexSize': 61440,
 'indexFreeStorageSize': 0,
 'fileSize': 0,
 'nsSizeMB': 0,
 'ok': 1}

In [ ]:
pipeline = [
    {"$group": {"_id": "$canal", "count": {"$sum": 1}}}
]

# Ejecuta la consulta
results = collection.aggregate(pipeline)

# Muestra los resultados
for result in results:
    print(f"Canal: {result['_id']}, Cantidad: {result['count']}")

Canal: Tengo un plan, Cantidad: 147
Canal: The Wild Project, Cantidad: 280
Canal: El Dotol Nastra, Cantidad: 373
Canal: WORLDCA$T, Cantidad: 134
Canal: PICPOD TV, Cantidad: 39
Canal: PATABAJO El Podcast, Cantidad: 163
Canal: News To You, Cantidad: 102
Canal: Uri Sabat, Cantidad: 87
Canal: Doctor Vic, Cantidad: 49
Canal: CHINOLO44, Cantidad: 6
Canal: Sin Miedo Al Éxito, Cantidad: 50
Canal: pepe&chema podcast, Cantidad: 93
Canal: Roberto Mz, Cantidad: 371
Canal: Somos Pueblo Media, Cantidad: 169
Canal: VOS PODÉS, EL PODCAST!, Cantidad: 27
